# FIKH Model: Creep Response

## Learning Objectives

1. Fit FIKH to **creep (step stress)** experimental data
2. Understand **delayed yielding** and viscosity bifurcation with fractional memory
3. Analyze how **alpha_structure** affects creep acceleration
4. Explore the transition from solid-like to liquid-like behavior
5. Compare fractional vs classical creep dynamics

## Prerequisites

- NB01: FIKH Flow Curve (concepts)
- Bayesian inference fundamentals

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~3-5 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~15-20 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FIKH

# Robust path resolution for execution from any directory
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    load_ml_ikh_creep,
    save_fikh_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
    get_fikh_param_names,
    plot_alpha_sweep,
    print_alpha_interpretation,
)

jax, jnp = safe_import_jax()
verify_float64()

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Fractional Creep

Creep response under constant applied stress reveals **delayed yielding** and **viscosity bifurcation**.

### Creep Regimes

1. **Elastic response** ($t \ll \tau$): $\gamma \approx \sigma/G$
2. **Delayed yielding**: Strain accelerates as structure breaks down
3. **Steady flow** ($t \gg \tau$): $\dot{\gamma} \to \dot{\gamma}_{ss}$

### Alpha Effect on Creep

- **Lower alpha**: Slower approach to yielding, stretched acceleration
- **Higher alpha**: Sharper yield transition (classical behavior)

## 3. Load Data

In [ ]:
# Load creep data from ML-IKH Excel
STRESS_PAIR = 0  # First stress jump (3 Pa → 5 Pa)
time_data, gamma_dot_data, sigma_init, sigma_final = load_ml_ikh_creep(stress_pair_index=STRESS_PAIR)

print(f"Data points: {len(time_data)}")
print(f"Time range: [{time_data.min():.4f}, {time_data.max():.2f}] s")
print(f"Shear rate range: [{gamma_dot_data.min():.4g}, {gamma_dot_data.max():.4g}] 1/s")
print(f"Stress jump: {sigma_init} Pa → {sigma_final} Pa")

In [ ]:
# Compute strain from shear rate (cumulative integration)
strain_data = np.cumsum(gamma_dot_data[:-1] * np.diff(time_data))
strain_data = np.insert(strain_data, 0, 0.0)

# Plot raw data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogy(time_data, gamma_dot_data, "ko", markersize=5)
ax1.set_xlabel("Time [s]", fontsize=12)
ax1.set_ylabel("Shear rate [1/s]", fontsize=12)
ax1.set_title(f"Creep Response: σ = {sigma_final} Pa", fontsize=13)
ax1.grid(True, alpha=0.3)

ax2.plot(time_data, strain_data, "ko", markersize=5)
ax2.set_xlabel("Time [s]", fontsize=12)
ax2.set_ylabel("Strain [-]", fontsize=12)
ax2.set_title("Cumulative Strain", fontsize=13)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
# Create and fit FIKH model
model = FIKH(include_thermal=False, alpha_structure=0.7)

t0 = time.time()
model.fit(time_data, strain_data, test_mode="creep", sigma_applied=sigma_final, method='scipy')
t_nlsq = time.time() - t0

param_names = get_fikh_param_names(include_thermal=False)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Compute fit quality and plot
strain_pred = model.predict(time_data, test_mode="creep", sigma_applied=sigma_final)
metrics = compute_fit_quality(strain_data, strain_pred)

print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g}")

time_fine = np.linspace(time_data.min(), time_data.max(), 300)
strain_pred_fine = model.predict(time_fine, test_mode="creep", sigma_applied=sigma_final)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, strain_data, "ko", markersize=5, alpha=0.7, label="Data")
ax.plot(time_fine, strain_pred_fine, "-", lw=2.5, color="C0", label="FIKH fit")
ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Strain [-]", fontsize=12)
ax.set_title(f"FIKH Creep Fit (R$^2$ = {metrics['R2']:.5f})", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Alpha Exploration

In [ ]:
# Alpha sweep for creep
alpha_values = [0.3, 0.5, 0.7, 0.9, 0.99]

fig = plot_alpha_sweep(
    model,
    protocol="creep",
    alpha_values=alpha_values,
    x_data=time_fine,
    sigma_applied=sigma_final,
    figsize=(14, 5),
)

fig.axes[0].plot(time_data, strain_data, "ko", markersize=3, alpha=0.5, label="Data")
fig.axes[0].legend(fontsize=8, loc="best")

display(fig)
plt.close(fig)

In [ ]:
# Physical interpretation
fitted_alpha = model.parameters.get_value("alpha_structure")
print_alpha_interpretation(fitted_alpha)

## 6. Bayesian Inference

In [ ]:
# Bayesian inference
initial_values = {name: model.parameters.get_value(name) for name in param_names}

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    time_data,
    strain_data,
    test_mode="creep",
    sigma_applied=sigma_final,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence
all_pass = print_convergence_summary(result, param_names)

In [ ]:
# Trace plots
idata = result.to_inference_data()
key_params = ["G", "sigma_y0", "tau_thix", "alpha_structure"]
axes = az.plot_trace(idata, var_names=key_params, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Key Parameters)", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Parameter comparison
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, param_names)

## 7. Save Results

In [ ]:
save_fikh_results(model, result, "fikh", "creep", param_names)
print("\nResults saved.")

## Key Takeaways

1. **Creep reveals delayed yielding** with viscosity bifurcation
2. **Lower alpha** → stretched creep acceleration, slower yielding
3. **Higher alpha** → sharper yield transition (classical)
4. **Creep data constrains yield parameters** (sigma_y0, delta_sigma_y)
5. **Combined protocols** (flow curve + creep) improve alpha estimation

### Next Steps

- **NB05**: SAOS (frequency response)
- **NB06**: LAOS (nonlinear oscillatory)